In [ ]:
import sys
import cv2
import numpy as np
import time

In [ ]:
#tracker = "BOOSTING"
tracker = "KCF"
#tracker = "CSRT"
# tracker = "MOSSE"
# tracker = "TLD"
#tracker = "MIL"
# tracker = "MEDIANFLOW"
# tracker = "GOTURN"

In [ ]:
"""Some variables initialization """
#Number of trials
Tot=1
#Video frames dimensions
frame_width=1920
frame_height=1080
#List of list of BB
BB_avg=[]   

In [ ]:
#Data for video cows
path="./Outputs/"
videoOut_name = path+"data_baseline_"+tracker+".avi"
videoIn_name = "./Inputs/cows.avi"
# Define an initial bounding box
BBS = (1120, 290, 260, 120)

In [ ]:
start_time = time.time()  
for num in range(Tot):
    #List to contain bounding box    
    BB_l=[]
    #Frame iteration
    it=0
    #Capture video where object tracking should be performed
    video = cv2.VideoCapture(videoIn_name)
    #Video output
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    video_output = cv2.VideoWriter(videoOut_name,fourcc,60,(frame_width,frame_height))
    #BB initialization
    BB=BBS
    
    if tracker == "BOOSTING":
        track = cv2.TrackerBoosting_create()
    elif tracker == "KCF":
        track = cv2.TrackerKCF_create()
    elif tracker == "MIL":
        track = cv2.TrackerMIL_create()
    elif tracker == "MOSSE":
        track = cv2.TrackerMOSSE_create()
    elif tracker == "TLD":
        track = cv2.TrackerTLD_create()
    elif tracker == "CSRT":
        track = cv2.TrackerCSRT_create()
    elif tracker == "MEDIANFLOW":
        track = cv2.TrackerMedianFlow_create()
    else :
        track = cv2.TrackerGOTURN_create()

    # Iterate through videos frames
    while True:
        # Read a new frame
        cont, frame = video.read()
        if not cont:
            break

        it+=1    
        # Initialize tracker with the first frame and bounding box
        if it ==1 and cont==True:
            cont = track.init(frame, BB)
            
        # Update the tracker
        cont, BB = track.update(frame)
        #Append new bounding box
        BB_l.append(BB)

        # Draw bounding box
        if cont:
            # In case track succeeds
            pt1 = (int(BB[0]), int(BB[1]))
            pt2 = (int(BB[0] + BB[2]), int(BB[1] + BB[3]))
            cv2.rectangle(frame, pt1, pt2, (255,0,0), 2, 1)
            # Show the type of the tracker
            cv2.putText(frame, "Tracker "+ tracker+ " Success " , (10,70), \
                    cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255,255,255),2);

        else :
            # In the opposite case
            cv2.putText(frame, "Tracker "+ tracker+ " Failure", (10,70),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.0,(0,0,255),2)


        if (num==Tot-1):
            video_output.write(frame)

        #if it==50:
        #    break
        
    BB_avg.append(BB_l)

In [ ]:
prog_duration= time.time() - start_time
prog_duration

In [ ]:
prog_duration/60

In [ ]:
prog_duration/(60*Tot)

In [ ]:
len(BB_l),len(BB_avg)

In [ ]:
BB_l_avg=[] 
BB_avg=np.array(BB_avg)
BB_l_avg=np.sum(BB_avg,0)/Tot
BB_l_avg

In [ ]:
#BB_l

In [ ]:
np.save(path+"BB_data_"+tracker+".npy",np.array(BB_l_avg))